In [194]:
from __future__ import absolute_import, division, print_function

In [195]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10

In [196]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [197]:
print(tf.__version__)
print(np.__version__)
print(matplotlib.__version__)

1.6.0-dev20180115
1.14.0
2.1.0


In [203]:
(features, labels), (X_test_orig, Y_test_orig) = cifar10.load_data()
features = features/255 - 0.5
labels = labels.flatten()

test_images = X_test_orig/255 - 0.5
test_labels = Y_test_orig.flatten()


In [204]:
features[0].shape

(32, 32, 3)

In [205]:
labels = labels.flatten()
labels

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [206]:
height = 32
width = 32
channels = 3
n_inputs = height * width

In [207]:
tf.reset_default_graph()

In [214]:
def initialize_parameters():
    height = 32
    width = 32
    channels = 3
    n_inputs = height * width
    X = tf.placeholder(tf.float32, shape=[None,  height, width, channels], name="X")
    dropout_rate = 0.3

    training = tf.placeholder_with_default(False, shape=(), name='training')
    X_drop = tf.layers.dropout(X, dropout_rate, training=training)
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    return X_drop, y, training

In [215]:
def get_next_batch(features, labels, train_size, batch_index, batch_size):
    training_images = features[:train_size,:,:]
    training_labels = labels[:train_size]
    
    start_index = batch_index * batch_size
    end_index = start_index + batch_size

    return features[start_index:end_index,:,:], labels[start_index:end_index]

In [216]:
def forward_propagation(X):
  
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=X,
      filters=32,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu)

    conv2 = tf.layers.conv2d(conv1, filters=64, 
                       kernel_size=3,
                       strides=2, padding="valid",
                       activation=tf.nn.relu, name="conv2")
    pool3 = tf.nn.max_pool(conv2,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 2, 2, 1],
                     padding="VALID")
    conv4 = tf.layers.conv2d(pool3, filters=128, 
                       kernel_size=4,
                       strides=3, padding="SAME",
                       activation=tf.nn.relu, name="conv4")

    pool5 = tf.nn.max_pool(conv4,
                     ksize=[1, 2, 2, 1],
                     strides=[1, 1, 1, 1],
                     padding="VALID")

    pool5_flat = tf.contrib.layers.flatten(pool5)

    fullyconn1 = tf.layers.dense(pool5_flat, 128,
                           activation=tf.nn.relu, name="fc1")

    fullyconn2 = tf.layers.dense(fullyconn1, 64,
                           activation=tf.nn.relu, name="fc2")
    logits = tf.layers.dense(fullyconn2, 10, name="output")
    return logits

In [222]:
tf.reset_default_graph()

n_epochs = 10
batch_size = 128
X, y, training = initialize_parameters()
logits = forward_propagation(X)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                      labels=y)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
#         batch_index = 0
        # Add this in when we want to run the training on all batches in CIFAR-10
        batch_index = 0

        train_size = int(len(features))

        for iteration in range(train_size // batch_size):
            X_batch, y_batch = get_next_batch(features, 
                                                                        labels, 
                                                                        train_size, 
                                                                        batch_index,
                                                                        batch_size)
            batch_index += 1

            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})

        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: test_images, y: test_labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        print ("Train Accuracy:", round(acc_train, 2)*100,"%",  end="\t")
        print ("Test Accuracy:", round(acc_test, 2)*100,"%", end="\n")
        model_saver = tf.train.Saver()
        save_path = model_saver.save(sess, "ADL/cifar_model")

0 Train accuracy: 0.5625 Test accuracy: 0.4986
Train Accuracy: 56.00000023841858 %	Test Accuracy: 50.0 %
1 Train accuracy: 0.671875 Test accuracy: 0.5598
Train Accuracy: 67.00000166893005 %	Test Accuracy: 56.00000023841858 %
2 Train accuracy: 0.6953125 Test accuracy: 0.6062
Train Accuracy: 69.9999988079071 %	Test Accuracy: 61.000001430511475 %
3 Train accuracy: 0.7109375 Test accuracy: 0.6295
Train Accuracy: 70.99999785423279 %	Test Accuracy: 62.99999952316284 %
4 Train accuracy: 0.734375 Test accuracy: 0.6338
Train Accuracy: 73.00000190734863 %	Test Accuracy: 62.99999952316284 %
5 Train accuracy: 0.7421875 Test accuracy: 0.6362
Train Accuracy: 74.00000095367432 %	Test Accuracy: 63.999998569488525 %
6 Train accuracy: 0.7578125 Test accuracy: 0.6284
Train Accuracy: 75.99999904632568 %	Test Accuracy: 62.99999952316284 %


KeyboardInterrupt: 

In [135]:
import tensorflow as tf
from tensorflow.python.tools import inspect_checkpoint as chkp

tf.reset_default_graph()
with tf.Session() as sess:
    # Restore variables from disk.
    saver = tf.train.import_meta_graph('ADL/cifar_model.meta')
    saver.restore(sess,"ADL/cifar_model")
    print("Model restored.")
    sess.run(accuracy, feed_dict={X: test_images, y: test_labels[0]})
    #print(predicition)

INFO:tensorflow:Restoring parameters from ADL/cifar_model
Model restored.


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("X:0", shape=(?, 32, 32, 3), dtype=float32) is not an element of this graph.

In [128]:
logits

<tf.Tensor 'output/BiasAdd:0' shape=(?, 10) dtype=float32>